<a href="https://colab.research.google.com/github/asya474/car_classification_neural_network/blob/main/final_project_data_investigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [195]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import time
import csv
import sys
import os
!pip install tensorflow_addons -q
import tensorflow_addons as tfa
import re
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

#print(os.listdir("/content/drive/MyDrive/Colab Notebooks/input"))


In [196]:
RANDOM_SEED = 42
VAL_SPLIT= 0.20 
mape=mean_absolute_percentage_error

In [197]:
zip_file='/content/drive/MyDrive/data.csv.zip'
z=zipfile.ZipFile(zip_file, 'r')
z.extractall()
print(os.listdir())

['.config', 'drive', 'data.csv', '__MACOSX', 'sample_data']


In [198]:
data=pd.read_csv('data.csv')

In [199]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [200]:
data.nunique()

status             159
private pool         1
propertyType      1280
street          337076
baths              229
homeFacts       321009
fireplace         1653
city              2026
schools         297365
sqft             25405
zipcode           4549
beds              1184
state               39
stories            348
mls-id           24907
PrivatePool          2
MlsId           232944
target           43939
dtype: int64

In [201]:
data.isna().sum()

status           39918
private pool    373004
propertyType     34733
street               2
baths           106338
homeFacts            0
fireplace       274070
city                34
schools              0
sqft             40577
zipcode              0
beds             91282
state                0
stories         150715
mls-id          352243
PrivatePool     336874
MlsId            66880
target            2481
dtype: int64

# target

In [202]:
data.target.value_counts()

$225,000     1462
$275,000     1355
$250,000     1312
$350,000     1296
$299,900     1276
             ... 
$390,359        1
$458,900+       1
274,359         1
$273,490+       1
$171,306        1
Name: target, Length: 43939, dtype: int64

In [203]:
data['new_target'] = data['target'].str.replace('+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [204]:
data['new_target'] = data['new_target'].str.replace('$', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [205]:
data['new_target'] = data['new_target'].str.replace(',', '')

In [206]:
data['new_target']=pd.to_numeric(data['new_target'], downcast='integer', errors='coerce')

In [207]:
data['new_target'].value_counts()

225000.0    1806
275000.0    1650
250000.0    1644
350000.0    1641
325000.0    1562
            ... 
218111.0       1
136466.0       1
236070.0       1
752911.0       1
171306.0       1
Name: new_target, Length: 34184, dtype: int64

In [208]:
data.drop(['target',], axis=1, inplace=True)

In [209]:
data.dropna(subset=['new_target'], inplace=True)

In [210]:
data['new_target'].describe()

count    3.743060e+05
mean     6.454072e+05
std      1.842196e+06
min      1.000000e+00
25%      1.877965e+05
50%      3.200000e+05
75%      5.845000e+05
max      1.950000e+08
Name: new_target, dtype: float64

In [211]:
#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

In [212]:
#import numpy as np
#import pandas as pd
#from pandas_profiling import ProfileReport

In [213]:
#profile = ProfileReport(data, title='data_report', html={'style':{'full_width':True}})

In [214]:
#profile.to_notebook_iframe()

In [215]:
#profile.to_file(output_file="data_report.html")

# status

In [216]:
data.status.value_counts()

for sale                   156100
Active                     105206
For sale                    43465
foreclosure                  5677
New construction             5459
                            ...  
Coming soon: Nov 8.             1
Coming soon: Oct 29.            1
Coming soon: Dec 15.            1
Pending W/Backup Wanted         1
Coming soon: Dec 23.            1
Name: status, Length: 155, dtype: int64

In [217]:
data.status.unique()

array(['Active', 'for sale', nan, 'New construction', 'New', 'For sale',
       'Pending', 'P', 'Active/Contingent', ' / auction',
       'Under Contract', 'Pre-foreclosure / auction',
       'Under Contract   Showing', 'Pre-foreclosure',
       'Under Contract Backups', 'Active Under Contract', 'foreclosure',
       'Foreclosed', 'Option Pending', 'Under Contract Show', 'Auction',
       'A Active', 'Contingent', 'Pending   Continue To Show',
       'Price Change', 'Back on Market', 'Active Option', 'Foreclosure',
       'Coming soon: Nov 21.', 'Contingent Finance And Inspection',
       'Coming soon: Dec 4.', 'P Pending Sale', 'Coming soon: Nov 23.',
       'Active With Contingencies', 'Pending Ab', 'Pf', 'Contingent Show',
       'Contract P', 'Contingent Take Backup', 'Apartment for rent',
       'Backup Contract', 'Option Contract', 'Pending Continue To Show',
       'pending', 'Pending Inspection', 'Active Option Contract', 'C',
       'Auction - Active', 'Contingent   Show', 'Pi

In [218]:
data['status'] = data.status.astype(str)
data['status'] = data.status.apply(lambda x: x.split(' ')[0].lower())

In [219]:
data.status.unique()

array(['active', 'for', 'nan', 'new', 'pending', 'p', 'active/contingent',
       '', 'under', 'pre-foreclosure', 'foreclosure', 'foreclosed',
       'option', 'auction', 'a', 'contingent', 'price', 'back', 'coming',
       'pf', 'contract', 'apartment', 'backup', 'c', 'pi', 'due', 'u',
       'lease/purchase', 'offer', 'listing', 'pending,', 'contingency',
       'condo', 'ct', 'temporary', 'closed', 'accepted', 'conditional',
       'accepting', 'conting', 'ps', 're', 'reactivated', 'uc'],
      dtype=object)

In [220]:
def transform_status(
    value: str
):  
    if isinstance(value, str):
        if  value in ['for']:
            return 'for sale'
        if  value in ['nan', '']:
            return 'not known'
        if  value in ['a', 'active/contingent']:
            return 'active'
        if  value in ['price']:
            return 'price change'
        if  value in ['back', 'backup']:
            return 'back on market'
        if  value in ['coming', 'c']:
            return 'coming soon'
        if value in ['p', 'pi', 'ps']:
            return 'pending'
        if value in ['u', 'uc']:
            return 'under'
        if value in ['re']:
            return 'reactivated'
        if value in ['ct']:
            return 'contract'
        if value in ['pre-foreclosure', 'foreclosure', 'foreclosed']:
            return 'foreclosure'
        if value in ['accepting', 'accepted']:
            return 'accepted'
        if value in ['contingency', 'conting']:
            return 'contingency'
        if value in ['condo', 'conditional']:
            return 'conditional'

    return value
data.status=data.status.apply(transform_status)

In [221]:
data.status.unique()

array(['active', 'for sale', 'not known', 'new', 'pending', 'under',
       'foreclosure', 'option', 'auction', 'contingent', 'price change',
       'back on market', 'coming soon', 'pf', 'contract', 'apartment',
       'due', 'lease/purchase', 'offer', 'listing', 'pending,',
       'contingency', 'conditional', 'temporary', 'closed', 'accepted',
       'reactivated'], dtype=object)

In [222]:
data['status'].describe()

count       374306
unique          27
top       for sale
freq        199567
Name: status, dtype: object

# private pool

In [223]:
data['private pool'].value_counts()

Yes    4151
Name: private pool, dtype: int64

In [224]:
data.drop(['private pool',], axis=1, inplace=True)

# propertyType

In [225]:
data['propertyType'].value_counts()

single-family home                                             91159
Single Family                                                  61888
Single Family Home                                             31725
condo                                                          25878
lot/land                                                       20532
                                                               ...  
1 Story, Contemporary, Other (See Remarks)                         1
Custom, Elevated, Other                                            1
Contemporary, Farmhouse                                            1
2 Stories, Traditional, Mediterranean, Texas Hill Country          1
Bilevel, Converted Dwelling, Loft with Bedrooms, Condo/Unit        1
Name: propertyType, Length: 1279, dtype: int64

In [226]:
data['propertyType'].unique()

array(['Single Family Home', 'single-family home', 'lot/land', ...,
       'Cabin/Lodge, Contemporary, Converted Barn, Cottage, Loft with Bedrooms, Log Home, Post & Beam',
       'Lake House, Single Detached, Tudor',
       'Bilevel, Converted Dwelling, Loft with Bedrooms, Condo/Unit'],
      dtype=object)

In [227]:
data['propertyType'] = data['propertyType'].astype(str)
data['propertyType'] = data['propertyType'].apply(lambda x: x.split(' ')[0].lower().replace(',', ''))

In [228]:
data['propertyType'].unique()
#требуется более точечная ифна с этим сериес,чем просто обрезка данных, тут и данные о стиле интерьера, и данные о колве комнат

array(['single', 'single-family', 'lot/land', 'townhouse', 'florida',
       'nan', 'coop', 'english', '2', 'multi-family', 'penthouse',
       'condo', 'land', 'condo/townhome/row', '', 'detached', '1',
       'other', 'colonial', 'transitional', 'high', 'mobile/manufactured',
       'tri-level', 'craftsman', 'federal', 'multi', 'traditional',
       'custom', 'cooperative', 'contemporary/modern', 'cape', 'mobile',
       'miscellaneous', 'mfd/mobile', 'bungalow', 'spanish/mediterranean',
       'contemporary', 'multi-level', 'condo/unit', 'ranch', 'low-rise',
       'rancher', 'urban', 'two', 'garden', 'farms/ranches', 'a-frame',
       'attached', 'one', 'georgian', 'dwelling', 'victorian',
       'apartment', 'cluster', 'manufactured', 'condominium',
       'condo/townhome', 'hi', 'mediterranean', 'cabin', 'bermuda',
       'split', 'farm', 'split-level', 'singlefamilyresidence', 'log',
       'bilevel', 'mountain', 'commercial/industrial', 'cottage',
       'high-rise', 'mid-rise'

In [229]:
def transform_status(
    value: str
):  
    if isinstance(value, str):
        if  value in ['nan', '']:
            return 'not known'
        if  value in ['1']:
            return 'one'
        if  value in ['2', '2.5', '2-story']:
            return 'two'
        if  value in ['3']:
            return 'back on market'
        if  value in ['coming', 'c']:
            return 'coming soon'
        if value in ['p', 'pi', 'ps']:
            return 'pending'
        if value in ['u', 'uc']:
            return 'under'
        if value in ['re']:
            return 'reactivated'
        if value in ['ct']:
            return 'contract'
        if value in ['pre-foreclosure', 'foreclosure', 'foreclosed']:
            return 'foreclosure'
        if value in ['accepting', 'accepted']:
            return 'accepted'
        if value in ['contingency', 'conting']:
            return 'contingency'
        if value in ['condo', 'conditional']:
            return 'conditional'

    return value
data.status=data.status.apply(transform_status)

In [230]:
data.isna().sum()

status               0
propertyType         0
street               2
baths           105288
homeFacts            0
fireplace       271878
city                34
schools              0
sqft             40111
zipcode              0
beds             90909
state                0
stories         149396
mls-id          349369
PrivatePool     334280
MlsId            64337
new_target           0
dtype: int64

# sreet

In [231]:
data.street.value_counts()

Address Not Disclosed               672
Undisclosed Address                 516
(undisclosed Address)               391
Address Not Available               175
Unknown Address                      72
                                   ... 
Lynmar Plan in Rough Hollow           1
MLS #: CORC5922640                    1
7784 Rosedale St                      1
Buildable plan: Residence 2 Plan      1
5983 Midcrown Dr                      1
Name: street, Length: 334447, dtype: int64

In [232]:
data['street']=data['street'].fillna(value='not known')

# baths

In [233]:
data.baths.value_counts()

2 Baths          51953
3 Baths          35356
2                20429
2.0              16355
4 Baths          14712
                 ...  
32                   1
5.25 Baths           1
41.0                 1
Bathrooms: 21        1
44.0                 1
Name: baths, Length: 225, dtype: int64

In [234]:
data['baths']=pd.to_numeric(data['baths'],  errors='coerce')

In [235]:
data['baths'].unique()

array([  3.5,   nan,   2. ,   3. ,   4. ,   1. ,   4.5,   5. ,   1.5,
         2.5,   8. ,   0. ,   7. ,   6. ,   5.5, 750. ,  10. ,  18. ,
         9. ,  12. ,   6.5,  14. ,  17. ,  16. ,   7.5,  11. ,  22. ,
        13. ,  32. ,  26. ,   9.5,   8.5,  35. ,  40. ,  29. ,  15. ,
        21. ,  20. ,  55. ,  41. ,  10.5,  27. ,  12.5,  24. ,  23. ,
        34. ,  64. ,  30. ,  44. ])

In [236]:
data['baths'].describe()

count    102741.000000
mean          4.249978
std          35.889117
min           0.000000
25%           2.000000
50%           2.000000
75%           3.000000
max         750.000000
Name: baths, dtype: float64

In [237]:
data['baths']=data['baths'].fillna(value=data['baths'].mean())

In [238]:
data['baths'].unique()

array([  3.5      ,   4.2499781,   2.       ,   3.       ,   4.       ,
         1.       ,   4.5      ,   5.       ,   1.5      ,   2.5      ,
         8.       ,   0.       ,   7.       ,   6.       ,   5.5      ,
       750.       ,  10.       ,  18.       ,   9.       ,  12.       ,
         6.5      ,  14.       ,  17.       ,  16.       ,   7.5      ,
        11.       ,  22.       ,  13.       ,  32.       ,  26.       ,
         9.5      ,   8.5      ,  35.       ,  40.       ,  29.       ,
        15.       ,  21.       ,  20.       ,  55.       ,  41.       ,
        10.5      ,  27.       ,  12.5      ,  24.       ,  23.       ,
        34.       ,  64.       ,  30.       ,  44.       ])

In [239]:
data['baths']=data['baths'].astype(float).apply(lambda x:round(x, 1))

In [240]:
data['baths'].unique()

array([  3.5,   4.2,   2. ,   3. ,   4. ,   1. ,   4.5,   5. ,   1.5,
         2.5,   8. ,   0. ,   7. ,   6. ,   5.5, 750. ,  10. ,  18. ,
         9. ,  12. ,   6.5,  14. ,  17. ,  16. ,   7.5,  11. ,  22. ,
        13. ,  32. ,  26. ,   9.5,   8.5,  35. ,  40. ,  29. ,  15. ,
        21. ,  20. ,  55. ,  41. ,  10.5,  27. ,  12.5,  24. ,  23. ,
        34. ,  64. ,  30. ,  44. ])

# homeFacts

In [241]:
data.homeFacts.value_counts()

{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': '—', 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                                                     7142
{'atAGlanceFacts': [{'factValue': None, 'factLabel': 'Year built'}, {'factValue': None, 'factLabel': 'Remodeled year'}, {'factValue': None, 'factLabel': 'Heating'}, {'factValue': None, 'factLabel': 'Cooling'}, {'factValue': None, 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                                          3484
{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factVa

# fireplace

In [242]:
data.fireplace.value_counts()

yes                                                                     49925
Yes                                                                     20619
1                                                                       14533
2                                                                        2432
Not Applicable                                                           1990
                                                                        ...  
Gas, Wood Burning, Two, Propane Logs Convey                                 1
Free-standing, Insert, Wood                                                 1
Wood Burning, Attached Fireplace Doors/Screen, Electric, Gas Starter        1
One, Living Room                                                            1
Ceiling Fan, SMAPL, Utility Connection, Walk-In Closets                     1
Name: fireplace, Length: 1652, dtype: int64

In [243]:
data.fireplace.isna().sum()

271878

In [247]:
data['fireplace']=data['fireplace'].apply(lambda x : x=='yes' if data['fireplace'].isna(x)=='False')

SyntaxError: ignored

In [ ]:
data.fireplace.isna().sum()

# city

In [ ]:
data.city.value_counts()

# schools

In [ ]:
data.schools.value_counts()

# sqft

In [ ]:
data.sqft.value_counts()

# zipcode

In [ ]:
data.zipcode.value_counts()

In [ ]:
data.zipcode.value_counts().plot.barh()

# beds

In [ ]:
data.beds.value_counts()

In [ ]:
data.beds.unique()

# state

In [ ]:
data.state.value_counts()

In [ ]:
data.state.value_counts().plot.barh()

# stories

In [ ]:
data.stories.value_counts()

# mls-id

In [ ]:
data['mls-id'].value_counts()

In [ ]:
data.drop(['mls-id',], axis=1, inplace=True)

# PrivatePool

In [ ]:
data.PrivatePool.value_counts()

In [ ]:
data['PrivatePool']=data['PrivatePool'].fillna(value='Not', inplace=True)

In [ ]:
data['PrivatePool'] = data['PrivatePool'].astype(str)
data['PrivatePool'] = data['PrivatePool'].apply(lambda x: x.lower())

In [ ]:
data['PrivatePool'].value_counts()

# MlsId

In [ ]:
data['MlsId'].value_counts()

In [ ]:
data['MlsId'] = pd.to_numeric(data['MlsId'], errors='coerce')

In [ ]:
data['MlsId'].value_counts()

In [ ]:
data['MlsId']=data['MlsId'].fillna(value=data['MlsId'].mean())

In [ ]:
data.isna().sum()